In [1]:
//Australia having very distinct climate zones,no single model will work for all australia
//Different models need to be trained for different locations
//In this approach,separate models will be created for different location and saved in the directory
//for the prediction particular location model will be loaded and prediction will be done
//Data used was weatherAUS_YN.csv(before imputing the  null values) as it was  observed  that 
//dropping null values are more effective than imputing (In our scenario imputing means blunting the features i.e.  generalising the data)
//Wind direction (categorical columns) not tested or used in the modelling as too much of encoding required and crashing the docker
//correlation matrix(wrangling file)& trial & Error methods were used to select the features)

Intitializing Scala interpreter ...

Spark Web UI available at http://6183b873be90:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591519064075)
SparkSession available as 'spark'


# Import required libraries

In [2]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
Logger.getLogger("org").setLevel(Level.ERROR)  

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator, VectorIndexer}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.log4j._


# Starting spark session

In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6bb3bb67


# Importing the file

In [4]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("/home/weatherAUS_YN.csv")

data: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [5]:
val df= data.withColumn("Month",month(data("Date")))

df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


#  Selecting required fields with right data type

In [6]:
val df1 = df.selectExpr("Date","Location",
                        //"cast(MinTemp as double) MinTemp",
                        "cast (MaxTemp as double) MaxTemp",
                       // "cast (Rainfall as double) Rainfall",
                       // "cast (Evaporation as double) Evaporation",
                        // "cast (Sunshine as double) Sunshine",
                        //"WindGustDir",
                        // "cast (WindGustSpeed as double) WindGustSpeed",
                        //"WindDir9am","WindDir3pm",
                         //"cast (WindSpeed9am as double) WindSpeed9am",
                         //"cast (WindSpeed3pm as double) WindSpeed3pm",
                         //"cast (Humidity9am as double) Humidity9am",
                          "cast (Humidity3pm as double) Humidity3pm", 
                         //"cast (Pressure9am as double) Pressure9am",
                         "cast (Pressure3pm as double) Pressure3pm",
                         //"cast (Cloud9am as double) Cloud9am",
                         //"cast (Cloud3pm as double) Cloud3pm",
                        "cast (Temp3pm as double) Temp3pm",
                        "cast (Temp9am as double) Temp9am",
                         "cast (RainToday as integer) RainToday",
                         "cast (Month as String) Month",
                         "cast (RainTomorrow as integer) RainTomorrow"
                      
                                                  )
                        

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [7]:
df1.columns

res2: Array[String] = Array(Date, Location, MaxTemp, Humidity3pm, Pressure3pm, Temp3pm, Temp9am, RainToday, Month, RainTomorrow)


# Filtering and forming a particular location data

In [8]:
val LocData=df1.filter($"Location"==="Sydney")toDF()

LocData: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [9]:
LocData.count()

res3: Long = 3337


# Dropping the null values

In [10]:
val Loc =LocData.na.drop()

Loc: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [11]:
Loc.count()

res4: Long = 3305


# Setting the label column(Rain tomorrow)

In [12]:
val df2 = (Loc.select(Loc("RainTomorrow").as("label"), 
           $"Location",
           $"Date",
          //$"MinTemp",
          $"MaxTemp", 
         $"Month",
         $"Raintoday",
         //$"WindDir3pm",
         //$"WindSpeed9am",
         // $"WindSpeed3pm", 
         //$"Humidity9am",
          $"Humidity3pm",
          //$"Pressure9am",
          //$"Cloud3pm", 
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

df2: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


# Indexing the month column(categorical)-To capture the seasonal pattern

In [13]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")




Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_4dc0a6d08cae


# Encoding the MonIn( column

In [14]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))


Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_99b67ff2d6e2


# Setting the assembler

In [15]:
 val assembler=(new VectorAssembler().setInputCols(Array(//"MinTemp",
                                                        "Raintoday",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                        //"Humidity9am",
                                                        //"WindSpeed3pm",
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f6f327b2501d


# Splitting the data into test and train sets

In [16]:
val Array(training,test)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


# Logistic regression

In [47]:
val lr=new LogisticRegression().setMaxIter(30).setThreshold(0.50)
       

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_0faf9be74669


# Setting the pipeline

In [48]:
val pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_661796b4f06c


# Training the model on training data

In [49]:
val model1=pipeline.fit(training)

model1: org.apache.spark.ml.PipelineModel = pipeline_661796b4f06c


# transforming test data with model

In [50]:
val results =model1.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


# Model Evaluation

In [51]:
val predictions=results.select($"prediction",$"label").as[(Double,Double)].rdd

predictions: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[438] at rdd at <console>:38


In [52]:
val metrics= new MulticlassMetrics(predictions)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@6abe84a1


In [53]:
println(metrics.confusionMatrix)

716.0  33.0   
126.0  131.0  


(1,1) no  pred no  rain
(2,2) yes pred yes rain
(1,2) no pred yes rain 
(2,1)  yes pred no rain 

In [54]:
metrics.accuracy

res18: Double = 0.841948310139165


# Saving the model

In [55]:
model1.write.overwrite().save("/home/model_Sydney")

# Saving the pipeline

In [56]:
pipeline.write.overwrite().save("/home/pipeline_Sydney")

# Reloading the model

In [57]:
val modelLoaded=PipelineModel.load("/home/model_Sydney")

modelLoaded: org.apache.spark.ml.PipelineModel = pipeline_661796b4f06c


# Using the model

In [58]:
val dfu= modelLoaded.transform(df2)

dfu: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [59]:
df2.show()

+-----+--------+-------------------+-------+-----+---------+-----------+-----------+-------+-------+
|label|Location|               Date|MaxTemp|Month|Raintoday|Humidity3pm|Pressure3pm|Temp9am|Temp3pm|
+-----+--------+-------------------+-------+-----+---------+-----------+-----------+-------+-------+
|    1|  Sydney|2008-02-01 00:00:00|   22.4|    2|        1|       84.0|     1017.4|   20.7|   20.9|
|    1|  Sydney|2008-02-02 00:00:00|   25.6|    2|        1|       73.0|     1016.4|   22.4|   24.8|
|    1|  Sydney|2008-02-03 00:00:00|   24.5|    2|        1|       86.0|     1015.6|   23.5|   23.0|
|    1|  Sydney|2008-02-04 00:00:00|   22.8|    2|        1|       90.0|     1011.8|   21.4|   20.9|
|    1|  Sydney|2008-02-05 00:00:00|   25.7|    2|        1|       74.0|     1004.8|   22.5|   25.5|
|    1|  Sydney|2008-02-06 00:00:00|   27.2|    2|        1|       62.0|      998.6|   23.8|   26.0|
|    1|  Sydney|2008-02-07 00:00:00|   26.3|    2|        1|       80.0|     1000.3|   21.7

In [60]:
val testev=dfu.select("Location","Date","RawPrediction","probability","label","prediction").toDF()

testev: org.apache.spark.sql.DataFrame = [Location: string, Date: timestamp ... 4 more fields]


In [61]:
testev.show()

+--------+-------------------+--------------------+--------------------+-----+----------+
|Location|               Date|       RawPrediction|         probability|label|prediction|
+--------+-------------------+--------------------+--------------------+-----+----------+
|  Sydney|2008-02-01 00:00:00|[-2.0041402762100...|[0.11876890447253...|    1|       1.0|
|  Sydney|2008-02-02 00:00:00|[-0.8975070067006...|[0.28956307756246...|    1|       1.0|
|  Sydney|2008-02-03 00:00:00|[-2.0135357891308...|[0.11778906017086...|    1|       1.0|
|  Sydney|2008-02-04 00:00:00|[-2.4777528474604...|[0.07743257910936...|    1|       1.0|
|  Sydney|2008-02-05 00:00:00|[-0.9609898951872...|[0.27668004514835...|    1|       1.0|
|  Sydney|2008-02-06 00:00:00|[-0.0372598516185...|[0.49068611460667...|    1|       1.0|
|  Sydney|2008-02-07 00:00:00|[-1.6607434764523...|[0.15966221932060...|    1|       1.0|
|  Sydney|2008-02-08 00:00:00|[-0.2783735293004...|[0.43085257204365...|    1|       1.0|
|  Sydney|